# Setup

In [ ]:
import os
import sys
sys.path.append("..")
debug = True # If True, will produce plots and/or verbose output to double-check
rerun_existing = False # 

# Path
print("Current working directory: ", sys.path[0])
%run -i "../parameters/parameters.py"
%run -i "../src/path.py"
%run -i "../src/setup.py"
%run -i "../src/functions.py"
%run -i "greedy_triangulation.py"
%run -i "graph_similarity.py"
%run -i "write_results_percentages.py"
%run -i "analyze_poi_based_results.py"
%run -i "generate_network_plots.py"

%load_ext watermark
%watermark -n -v -m -g -iv

In [ ]:

city = "bern"
iterations = 2

prune_quantiles = [0.5]
subgraph_percentages_tests = [
    [0.25, 0.25, 0.25, 0.25],
    [0.75, 0.25],
    [0.25, 0.75],
    [0.33, 0.33, 0.34],
]

prune_measures = ["betweenness"]

# Run the program

In [ ]:
placeid = city
print(placeid + ": Generating graph and POIs")

G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
    nnids = [int(line.rstrip()) for line in f]

for prune_measure in prune_measures:
    print(f"Prune measure: {prune_measure}")

    (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

    for subgraph_percentages in subgraph_percentages_tests:
        subgraph_percentage_string = '_'.join(map(str, subgraph_percentages))
        file_name = f"{placeid}_{prune_measure}_{subgraph_percentage_string}.csv"
        print(f"Writing data to {file_name}")
        dir = "csv_data/"
        path = os.path.join(dir, file_name)
        with open(path, 'w') as f:
            writer = csv.writer(f)
            writer.writerow([
                "sum_of_errors",
                "continuous_vertices",
                "bikengrowth_vertices",
                "continuous_disconnected_pairs",
                "bikengrowth_disconnected_pairs",
            ])

            for i in tqdm(range(iterations), desc="Iterations", leave=False):
                if prune_measure == "random":
                    (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

                gts, abstract_gts = greedy_triangulation_in_steps(G_carall, nnids, subgraph_percentages, prune_quantiles, prune_measure)
                (MST, MST_abstract) = mst_routing(G_carall, nnids)
                for prune_quantile, gt, szell_gt in zip(prune_quantiles, gts, GTs):
                    error, our_disconnected_points, szell_disconnected_points = sum_of_errors_pois(szell_gt, gt, nnids)

                writer.writerow([error, gt.vcount(), szell_gt.vcount(), our_disconnected_points, szell_disconnected_points])
                results = {"placeid": placeid, "prune_measure": prune_measure, "poi_source": poi_source, "prune_quantiles": prune_quantiles, "GTs": GTs, "GT_abstracts": GT_abstracts, "MST": MST, "MST_abstract": MST_abstract}
                write_result_with_percentages(res=results, mode="pickle", placeid=placeid, iteration=str(i), subgraph_percentage=subgraph_percentage_string, poi_source=poi_source, prune_measure=prune_measure, suffix=".pickle")

In [ ]:
string_list_subgraph_percentage = []
for subgraph_percentage in subgraph_percentages_tests:
    string_list_subgraph_percentage.append('_'.join(map(str, subgraph_percentage)))



In [ ]:
# Analyze results
analyze_existing_infrastructure([placeid])

analyze_poi_base_results_iterations(placeid=placeid, subgraphs_percentages=string_list_subgraph_percentage, iterations=iterations)

In [ ]:
#generate_plots
existing_network_plot(placeid=placeid, prune_measures=prune_measures)

In [ ]:
%run -i "generate_network_plots.py"

subgraph_percentages_with_iterations_plot(placeid=placeid, prune_measures=prune_measures, subgraphs_percentages=string_list_subgraph_percentage, iterations=iterations)